In [ ]:
import pandas as pd

# Import task data
task_data = pd.read_csv("Data\\onet_tasks.csv")
task_data["isco08_1dig"] = task_data["isco08"].astype(str).str[:1].astype(int)
aggdata = task_data.groupby(["isco08_1dig"]).mean()
aggdata = aggdata.drop(columns=["isco08"])

# Import employment data for Belgium, Spain, and Poland
countries = ["Belgium", "Spain", "Poland"]
employment_data = pd.concat([pd.read_excel("Data\\Eurostat_employment_isco.xlsx", sheet_name=f"ISCO{i}") for i in range(1, 10)])
total_employment = employment_data[countries].sum(axis=1)

# Merge employment and task data
merged_data = pd.merge(employment_data, aggdata, left_on="isco08", right_index=True, how="left")
merged_data["total_employment"] = total_employment
merged_data[countries] = merged_data[countries].div(merged_data["total_employment"], axis=0)

# Calculate intensity score
intensity_score = merged_data["data_value"] * merged_data[countries].mean(axis=1)
intensity_score = intensity_score.reset_index().rename(columns={"isco08_1dig": "Occupation"}).groupby("Occupation")["data_value"].sum()
